Goal of this script is to generate, for each image in the test set, a multicut compressed version using the quality levels  
1, 25, and 50. The decoded images are then saved in directory 

**./samples/multicut/{quality}/{category}/{filename}.png**

For the algorithm, we exclusively select the ensemble codec with:
- the ensemble multicut codec
- the differential mean partition codec
- zlib compression activated
- the greedy grid optimizer

In [1]:
from mlcv_py import make_mask_with_size, encode_mask_with_size
from mlcv_py import MULTICUT_CODEC as mc, PARTITION_CODEC as pc, OPTIMIZER as opt
import cv2
from pathlib import Path
import shutil
from tqdm import tqdm

In [2]:
QUALITY_LEVELS = [1, 25, 50]
SRC_ROOT = Path("../data/splitimages/test")

quality_dirs = dict()
for quality in QUALITY_LEVELS:
    out_dir = Path(f"samples/multicut/{int(quality)}/")
    if out_dir.exists():
        shutil.rmtree(str(out_dir))
    out_dir.mkdir()
    quality_dirs[quality] = out_dir

for category_dir in tqdm(SRC_ROOT.glob("*")):
    if not category_dir.is_dir(): continue

    for img_path in category_dir.glob("*.png"):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
        for quality in QUALITY_LEVELS:
            target = quality_dirs[quality] / category_dir.name
            target.mkdir(exist_ok=True)

            mask, _ = make_mask_with_size(img, mc.ENSEMBLE, pc.DIFFERENTIAL, opt.GREEDY_GRID, quality)
            decoded_img, total_size = encode_mask_with_size(img, mask, mc.ENSEMBLE, pc.DIFFERENTIAL, True, quality)
            target = target / f"{total_size}###{img_path.name}"

            cv2.imwrite(str(target), decoded_img)

13it [34:00, 156.99s/it]
